In [1]:
import torch
from torch import nn
import numpy as np
from mapsgan import TrajectoryDataset
import sys
%load_ext autoreload
%autoreload 2

## How depends the shape of hidden on the batchsize in LSTM?

In [46]:
num_classes = 3
input_size = 4
hidden_size = 7
batch_size =  5
sequence_length = 12
num_layers =  2
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers) # input_sze, hidden_size, batch_first
    
    def forward(self, x, hidden):
        x = x.view(sequence_length, batch_size, input_size) # batch_size, seq_len, input_size
        out, hidden = self.rnn(x, hidden)
        out = out.view(-1, num_classes) # num_classes is 5
        return out, hidden
    
    def init_hidden(self):
        return (torch.zeros(num_layers, batch_size, hidden_size),
                torch.zeros(num_layers, batch_size, hidden_size))
                

In [47]:
model = Model()
input = torch.rand(sequence_length, batch_size, input_size)
hidden = model.init_hidden()
out, hidden = model(input, hidden)

In [48]:
hidden[0].shape #(num_layers, batch_size, hidden_size)

torch.Size([2, 5, 7])

## Toy Example
Predict ihello

In [2]:
word = 'hihello'
letters = np.unique([x for x in word])
lookup = {let:num for num, let in enumerate(letters)}
idx2char = {num:let for num, let in enumerate(letters)}

In [3]:
vec = {l:np.zeros(letters.size) for l in letters}
for i, (letter,val) in enumerate(vec.items()):
    vec[letter][i]=1

In [4]:
lookup

{'e': 0, 'h': 1, 'i': 2, 'l': 3, 'o': 4}

In [5]:
x = 'hihell'
x_onehot = torch.Tensor([[vec[let] for let in x]])
y = 'ihello'
labels = torch.Tensor([[lookup[l]] for l in y])

In [6]:
x_onehot = torch.cat([x_onehot, torch.zeros_like(x_onehot)], dim = 1)

In [7]:
labels = torch.cat([labels, torch.zeros_like(labels)-1], dim = 0)

In [8]:
labels

tensor([[ 2.],
        [ 1.],
        [ 0.],
        [ 3.],
        [ 3.],
        [ 4.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.]])

In [9]:
x_onehot.shape

torch.Size([1, 12, 5])

In [10]:
num_classes, input_size, hidden_size, batch_size, sequence_length, num_layers = 5, 5, 5, 1, 12, 1
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first = True) # input_sze, hidden_size, batch_first
    
    def forward(self, x, hidden):
        x = x.view(batch_size, sequence_length, input_size) # batch_size, seq_len, input_size
        out, hidden = self.rnn(x, hidden)
        out = out.view(-1, num_classes) # num_classes is 5
        return hidden, out
    
    def init_hidden(self):
        return torch.zeros(num_layers, batch_size, hidden_size)

In [282]:
model = Model()

In [283]:
out, hidden = model(x_onehot, model.init_hidden())

In [284]:
out, hidden

(tensor([[[ 0.7530, -0.1349,  0.2548,  0.2163,  0.4934]]], grad_fn=<ViewBackward>),
 tensor([[ 0.4792,  0.0799,  0.6758,  0.3247,  0.7181],
         [ 0.7376, -0.1749,  0.5956,  0.3965,  0.5270],
         [ 0.3284, -0.2711,  0.5929,  0.6146,  0.5298],
         [ 0.5054,  0.0884,  0.7353, -0.0268,  0.2535],
         [ 0.6318, -0.0575,  0.1240,  0.2851,  0.4715],
         [ 0.7530, -0.1349,  0.2548,  0.2163,  0.4934]],
        grad_fn=<ViewBackward>))

model(x_on

In [361]:
model = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first = True)
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 0.1)
hidden = torch.zeros(num_layers, batch_size, hidden_size)

for epoch in range(100):
    optim.zero_grad()
    loss = 0
    output, hidden = model(x_onehot, hidden)
    output = output.view(sequence_length, -1)
    #print([idx2char(int(torch.max(hid, 1)[1])] for hid in hidden])
    loss += criterion(output, labels.squeeze().long())
    
#     for input, label in zip(x_onehot, labels):
#         hidden, output = model(input, hidden)
#         _, pred = torch.max(output, 1)
#         print(idx2char[int(pred)])
#         loss  += criterion(output, label.long())
        
    print("epoch: %d, loss: %1.3f"%(epoch+1, loss.data[0]))
    loss.backward(retain_graph=True)
    optim.step()


RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at /opt/conda/conda-bld/pytorch-cpu_1532578932944/work/aten/src/THNN/generic/ClassNLLCriterion.c:93

In [350]:
[idx2char[int(torch.argmax(l, dim = 0))] for l in output], [idx2char[int(l)] for l in labels.squeeze()]

(['e', 'h', 'e', 'l', 'l', 'o', 'e', 'e', 'e', 'e', 'e', 'e'],
 ['i', 'h', 'e', 'l', 'l', 'o', 'e', 'e', 'e', 'e', 'e', 'e'])

In [261]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

# Number of seq_len is len(example)-1 == time steps
seq_len = 6 # |hihell|==6
# Number of batches
batch_size = 1
# Number of features
input_size = 5 
# Hidden size, output size
hidden_size = 5   
# Number of layers
num_layer=1
# One hot encoding for each char in 'h','i','e','l','o'
h = [1, 0, 0, 0, 0]
i = [0, 1, 0, 0, 0]
e = [0, 0, 1, 0, 0]
l = [0, 0, 0, 1, 0]
o = [0, 0, 0, 0, 1]

idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hihell -> ihello

# The input need to match this format: (seg_len, batch_size, input_size)
input = torch.tensor([h, i, h, e, l, l], dtype = torch.float)
input = input.view(seq_len, batch_size, input_size)

# Same with target (1, batch_size, hidden_size)
y_data = [1, 0, 2, 3, 3, 4]    # ihello
target = torch.tensor(y_data)
#print(input.size(), target.size())

# Create random hidden state
state = torch.randn(num_layer, batch_size, hidden_size)

# Create RNN 
rnn = nn.RNN(input_size=input_size, 
             hidden_size=hidden_size, 
             num_layers=1, 
             nonlinearity='tanh')

# Use optim
parameters = rnn.parameters()
optimizer = torch.optim.Adam(parameters, lr = 1e-2)
# Loss function
criterion = torch.nn.CrossEntropyLoss()

loss_value = []
iteration = []
# Train
for epoch in range(250):
    out,state = rnn(input,state)
    out = out.view(seq_len, -1)
    res =  [idx2char[x] for x in torch.argmax(out, dim=1)]
    loss = criterion(out,target)
    # Print result occasionally
    if epoch%10 == 0:
        print('epoch = ', epoch)
        print('loss = ', loss.item())
        print(res)
        print('---------------------\n')
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
    loss_value.append(loss)
    iteration.append(epoch)

epoch =  0
loss =  1.5305711030960083
['i', 'h', 'i', 'o', 'o', 'o']
---------------------

epoch =  10
loss =  1.5292962789535522
['h', 'o', 'h', 'l', 'o', 'o']
---------------------

epoch =  20
loss =  1.332958459854126
['i', 'h', 'i', 'l', 'o', 'o']
---------------------

epoch =  30
loss =  1.1438624858856201
['i', 'h', 'e', 'l', 'l', 'l']
---------------------

epoch =  40
loss =  0.9774511456489563
['i', 'h', 'e', 'l', 'l', 'o']
---------------------

epoch =  50
loss =  0.8569862842559814
['i', 'h', 'e', 'l', 'l', 'o']
---------------------

epoch =  60
loss =  0.775343120098114
['i', 'h', 'e', 'l', 'l', 'o']
---------------------

epoch =  70
loss =  0.7400549054145813
['i', 'h', 'e', 'l', 'l', 'o']
---------------------

epoch =  80
loss =  0.8040781617164612
['i', 'h', 'e', 'l', 'l', 'o']
---------------------

epoch =  90
loss =  0.8265789151191711
['i', 'h', 'e', 'l', 'l', 'o']
---------------------

epoch =  100
loss =  0.8235683441162109
['i', 'h', 'e', 'l', 'l', 'o']
--

## Trajectory Toy Example